## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
import numpy as np


### Collect Data

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [5]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [7]:
data.drop(["date","symbol"],axis=1,inplace=True)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [9]:
data=data[0:1000]

In [10]:
data["volume"]=data["volume"]/1000000


In [11]:
features=data.drop(["volume"],axis=1)

In [12]:
target=data["volume"]

### Divide the data into train and test sets

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.30)

#### Convert Training and Test Data to numpy float32 arrays


In [14]:
X_train=X_train.astype("float32")

In [15]:
X_test=X_test.astype("float32")

In [16]:
y_train=y_train.astype("float32")

In [17]:
y_test=y_test.astype("float32")

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [18]:
from sklearn.preprocessing import Normalizer

In [19]:
normalizer=Normalizer()

In [20]:
X_train=normalizer.transform(X_train)

In [21]:
X_test=normalizer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [22]:
X_train.shape

(700, 4)

In [23]:
y_train.shape

(700,)

In [24]:
y_train=np.array([y_train]).T

In [25]:
y_train.shape

(700, 1)

In [26]:
y_test=np.array([y_test]).T

In [27]:
y_test.shape

(300, 1)

In [28]:
#Instead of initializing the weights and bias to zeros, I am initializing with the random numbers

In [34]:
session=tf.Session()

In [33]:
w=tf.Variable(tf.random_normal(shape=[4,1]))
b=tf.Variable(tf.random_normal(shape=[1,1]))

2.Define a function to calculate prediction

In [71]:
#I am following my way to do the model prediction and evaluating the Model

In [35]:
X=tf.placeholder(name="X",shape=[None,4],dtype=tf.float32)

In [36]:
y=tf.placeholder(name="y",shape=[None,1],dtype=tf.float32)

In [37]:
output=tf.add(tf.matmul(X,w),b)

In [38]:
loss=tf.reduce_mean(tf.squared_difference(y,output))

In [49]:
batch_size=30

In [39]:
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.02)

In [40]:
train=optimizer.minimize(loss)

In [41]:
init=tf.global_variables_initializer()
session.run(init)

In [57]:
for i in range(100):
    random_indices=np.random.choice(len(X_train),size=batch_size)
    x_data=X_train[random_indices]
    y_data=y_train[random_indices]
    session.run(train,feed_dict={X:x_data,y:y_data})
    print("w is:",session.run(w),\
         "\nb is:",session.run(b),\
         "\nloss:",session.run(loss,feed_dict={X:x_data,y:y_data}))
    print("\n")

w is: [[ 2.4831676 ]
 [-0.2243042 ]
 [ 0.87348723]
 [-0.49836338]] 
b is: [[3.8589706]] 
loss: 22.241512


w is: [[ 2.4550436 ]
 [-0.25235844]
 [ 0.84555936]
 [-0.5267071 ]] 
b is: [[3.8027444]] 
loss: 11.463899


w is: [[ 2.51068   ]
 [-0.19644997]
 [ 0.9001979 ]
 [-0.47024885]] 
b is: [[3.91408]] 
loss: 210.10483


w is: [[ 2.4596086 ]
 [-0.24749777]
 [ 0.84956604]
 [-0.5218063 ]] 
b is: [[3.8119216]] 
loss: 10.308175


w is: [[ 2.4554458 ]
 [-0.25167537]
 [ 0.84518945]
 [-0.52588063]] 
b is: [[3.8035285]] 
loss: 30.512056


w is: [[ 2.4595327 ]
 [-0.24757914]
 [ 0.849309  ]
 [-0.5216145 ]] 
b is: [[3.8118148]] 
loss: 88.13758


w is: [[ 2.4594367 ]
 [-0.24719578]
 [ 0.8491038 ]
 [-0.52101094]] 
b is: [[3.812172]] 
loss: 35.345776


w is: [[ 2.459515  ]
 [-0.24617825]
 [ 0.8493292 ]
 [-0.51990527]] 
b is: [[3.813404]] 
loss: 23.386515


w is: [[ 2.469236  ]
 [-0.23686334]
 [ 0.8580864 ]
 [-0.5099434 ]] 
b is: [[3.8323038]] 
loss: 42.152386


w is: [[ 2.486689  ]
 [-0.21942371]
 [ 0.8

In [58]:
w=session.run(w)

In [60]:
w

array([[ 2.4082735],
       [-0.3027224],
       [ 0.7669201],
       [-0.5495263]], dtype=float32)

In [61]:
b=session.run(b)

In [62]:
b

array([[3.7044003]], dtype=float32)

In [63]:
#Model Evaluation
train_predicted=session.run(output,feed_dict={X:X_train})

In [64]:
train_predicted

array([[4.858347 ],
       [4.8635526],
       [4.8504095],
       ...,
       [4.8670807],
       [4.858364 ],
       [4.848156 ]], dtype=float32)

In [65]:
train_mse=session.run(loss,feed_dict={X:X_train,y:y_train})

In [66]:
train_mse

156.66138

In [67]:
test_predicted=session.run(output,feed_dict={X:X_test})

In [68]:
test_mse=session.run(loss,feed_dict={X:X_test,y:y_test})

In [69]:
test_mse

155.93457

In [102]:
session.close()

3.Loss (Cost) Function [Mean square error]

In [70]:
loss=tf.reduce_mean(tf.squared_difference(y,output))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [72]:
#Following the approach taught in the class to predict the model

In [50]:
w = tf.zeros(shape=[4,1])
b = tf.zeros(shape=[1,1])

In [51]:
def prediction(x, w, b):    
    return(tf.add(tf.matmul(x, w),b))

In [52]:
def loss(y_actual, y_predicted):    
    return(tf.reduce_mean(tf.squared_difference(y_actual,y_predicted)))

In [53]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        #print(w)
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
        #print("current prediction:",current_prediction.numpy())
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [55]:
loss_every_5th_iteration=[]

In [59]:
for i in range(100):    
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    loss_value=loss(y_train,prediction(X_train,w,b)).numpy()
    print('Current Loss on iteration', i,loss_value)
    if i%5==0:
          loss_every_5th_iteration.append(loss_value)
        


Current Loss on iteration 0 161.60574
Current Loss on iteration 1 161.60576
Current Loss on iteration 2 161.60571
Current Loss on iteration 3 161.60571
Current Loss on iteration 4 161.60571
Current Loss on iteration 5 161.60568
Current Loss on iteration 6 161.60565
Current Loss on iteration 7 161.60562
Current Loss on iteration 8 161.60562
Current Loss on iteration 9 161.60558
Current Loss on iteration 10 161.60558
Current Loss on iteration 11 161.60558
Current Loss on iteration 12 161.60555
Current Loss on iteration 13 161.60555
Current Loss on iteration 14 161.60556
Current Loss on iteration 15 161.60553
Current Loss on iteration 16 161.60553
Current Loss on iteration 17 161.60551
Current Loss on iteration 18 161.60548
Current Loss on iteration 19 161.60547
Current Loss on iteration 20 161.60544
Current Loss on iteration 21 161.60542
Current Loss on iteration 22 161.60544
Current Loss on iteration 23 161.60541
Current Loss on iteration 24 161.60538
Current Loss on iteration 25 161.60

In [60]:
loss_every_5th_iteration

[161.61401,
 161.61166,
 161.61006,
 161.60902,
 161.60828,
 161.60776,
 161.60738,
 161.6071,
 161.60687,
 161.60672,
 161.60661,
 161.60649,
 161.60638,
 161.60634,
 161.6062,
 161.60614,
 161.60606,
 161.60599,
 161.60588,
 161.6058,
 161.60574,
 161.60568,
 161.60558,
 161.60553,
 161.60544,
 161.60536,
 161.60529,
 161.60522,
 161.60516,
 161.60509,
 161.60498,
 161.6049,
 161.60484,
 161.60478,
 161.6047,
 161.60461,
 161.60454,
 161.60446,
 161.60439,
 161.60431]

In [46]:
tf.executing_eagerly()

True

### Get the shapes and values of W and b

In [63]:
w=w.numpy()

In [64]:
b=b.numpy()

In [65]:
w.shape

(4, 1)

In [66]:
b.shape

(1, 1)

### Model Prediction on 1st Examples in Test Dataset

In [68]:
prediction(X_test[0:1],w,b).numpy()

array([[5.076215]], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [68]:
iris=pd.read_csv("iris.csv")

In [69]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [70]:
iris.isna().any()

Id               False
SepalLengthCm    False
SepalWidthCm     False
PetalLengthCm    False
PetalWidthCm     False
Species          False
dtype: bool

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [71]:
iris_dummy=pd.get_dummies(data=iris["Species"])


In [72]:
iris_dummy

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [73]:
iris_df=pd.concat([iris,iris_dummy.iloc[:,0:3]],axis=1)


In [74]:
iris_df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,5.1,3.5,1.4,0.2,Iris-setosa,1,0,0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,1,0,0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,1,0,0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,1,0,0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,1,0,0
5,6,5.4,3.9,1.7,0.4,Iris-setosa,1,0,0
6,7,4.6,3.4,1.4,0.3,Iris-setosa,1,0,0
7,8,5.0,3.4,1.5,0.2,Iris-setosa,1,0,0
8,9,4.4,2.9,1.4,0.2,Iris-setosa,1,0,0
9,10,4.9,3.1,1.5,0.1,Iris-setosa,1,0,0


### Splitting the data into feature set and target set

In [75]:
iris_df.replace(to_replace="Iris-setosa",value=0,inplace=True)
iris_df.replace(to_replace="Iris-versicolor",value=1,inplace=True)
iris_df.replace(to_replace="Iris-virginica",value=2,inplace=True)


In [76]:
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,5.1,3.5,1.4,0.2,0,1,0,0
1,2,4.9,3.0,1.4,0.2,0,1,0,0
2,3,4.7,3.2,1.3,0.2,0,1,0,0
3,4,4.6,3.1,1.5,0.2,0,1,0,0
4,5,5.0,3.6,1.4,0.2,0,1,0,0


In [77]:
features=iris_df[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]]
target=iris_df["Species"]

In [83]:
X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.30)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [78]:
model = tf.keras.Sequential()


In [79]:
#Add the Dense Layer for Prediction
model.add(tf.keras.layers.Dense(3, input_shape=(4,),activation="softmax"))


In [80]:
#For Normalization
model.add(tf.keras.layers.BatchNormalization())

In [81]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [82]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 3)                 15        
_________________________________________________________________
batch_normalization_1 (Batch (None, 3)                 12        
Total params: 27
Trainable params: 21
Non-trainable params: 6
_________________________________________________________________


In [85]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

In [86]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

W1103 16:04:52.789997  5928 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 10ms/sample - loss: 7.7105 - acc: 0.3333 - val_loss: 4.5466 - val_acc: 0.3778
Epoch 2/100
105/105 [==============================] - 0s 114us/sample - loss: 10.9958 - acc: 0.5048 - val_loss: 4.5764 - val_acc: 0.3778
Epoch 3/100
105/105 [==============================] - 0s 114us/sample - loss: 10.3928 - acc: 0.5524 - val_loss: 4.5855 - val_acc: 0.3778
Epoch 4/100
105/105 [==============================] - 0s 114us/sample - loss: 10.6728 - acc: 0.5810 - val_loss: 4.5907 - val_acc: 0.3778
Epoch 5/100
105/105 [==============================] - 0s 124us/sample - loss: 10.4794 - acc: 0.6095 - val_loss: 4.5883 - val_acc: 0.3778
Epoch 6/100
105/105 [==============================] - 0s 152us/sample - loss: 10.9281 - acc: 0.6095 - val_loss: 4.5868 - val_acc: 0.3778
Epoch 7/100
105/105 [==============================] - 0s 104us/sample - loss: 10.7602 - acc: 0.6286 - val_loss: 4.5826 - val_acc

### Model Prediction

In [88]:
prediction = model.predict(X_test[0:1])


In [89]:
prediction

array([[ 0.37937382,  0.01876831, -0.27195364]], dtype=float32)

In [90]:
y_test[0:1]

array([[1., 0., 0.]], dtype=float32)

### Save the Model

In [87]:
model.save('iris_model.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?